# HelioForge & CosmoView
## Overview, Tutorial, and Demonstration

This notebook provides a structured overview of the **HelioForge** simulation library
and the **CosmoView** visualization tool.

It is suitable both for:
- a short technical presentation (≈10–15 minutes),
- independent study as a guided tutorial.

The content progresses from basic concepts to more advanced usage.


---
## 1. Conceptual overview

### HelioForge

HelioForge is a minimal heliocentric simulation framework based on a two-body
approximation.

Core characteristics:
- circular, coplanar orbits,
- explicit time stepping,
- deterministic behavior,
- clarity over physical completeness.

HelioForge provides:
- data models (`CentralBody`, `Planet`),
- system container (`SolarSystem`),
- time evolution (`Simulation`),
- Kepler helpers (`Kepler`),
- procedural generation and presets,
- JSON export and import.

### CosmoView

CosmoView is a pygame-based real-time viewer for HelioForge simulations.

It renders:
- orbital paths,
- planet motion,
- simulation time progression,

and provides interactive controls for navigation and speed.


---
## 2. Imports and setup


In [ ]:

from helioforge import CentralBody, SolarSystem, make_solar_system
from helioforge.models import Planet
from helioforge.generation import generate_planets
from helioforge.kepler import Kepler
from helioforge.simulation import Simulation
from helioforge.constants import AU_M, DAY_S


---
## 3. Minimal generated system

This example demonstrates the simplest complete HelioForge workflow:
- create a star,
- generate planets procedurally,
- step the simulation,
- inspect positions.


In [ ]:

star = CentralBody(
    name="DemoStar",
    mass_kg=1.0e30,
    radius_m=7.0e8,
    luminosity_w=0.0,
)

planets = generate_planets(star, n_planets=3, seed=1)
system = SolarSystem(star, planets)
sim = Simulation(system)

sim.step(DAY_S)
system.state_m()


### Time stepping

Simulation time advances explicitly. Positions update deterministically
based on orbital period and phase.


In [ ]:

for day in range(3):
    sim.step(DAY_S)
    print(f"Day {day + 1}")
    for name, (x, y) in sim.system.state_m().items():
        print(f"  {name}: x={x:.2e} m, y={y:.2e} m")


---
## 4. Solar System preset

HelioForge includes an approximate Solar System preset.
It provides a familiar reference system for demonstrations.


In [ ]:

sun, planets_solar = make_solar_system()
system_solar = SolarSystem(sun, planets_solar)

system_solar.step(DAY_S)
[p.name for p in system_solar.planets]


---
## 5. Procedural generation in detail

Procedural generation allows control over:
- number of planets,
- orbital bounds,
- deterministic randomness.


In [ ]:

planets_proc = generate_planets(
    star,
    n_planets=6,
    seed=42,
    inner_au=0.5,
    outer_au=25.0,
)

for p in planets_proc:
    print(f"{p.name:8} | {p.kind:10} | {p.distance_m / AU_M:.2f} AU")


---
## 6. Building a system manually

Systems can be constructed explicitly without generators.
This provides full control over orbital layout and parameters.


In [ ]:

import math

star_hand = CentralBody(
    name="HandMadeStar",
    mass_kg=1.9885e30,
    radius_m=6.9634e8,
    luminosity_w=0.0,
)

k = Kepler(star_hand.mass_kg)

specs = [
    ("Alpha", "rocky",     1.0 * 5.972e24,   1.0 * 6.371e6,   0.7),
    ("Beta",  "rocky",     0.8 * 5.972e24,   0.9 * 6.371e6,   1.0),
    ("Gamma", "gas_giant", 0.6 * 1.898e27,   1.0 * 6.9911e7,  5.0),
]

planets_hand = []
for i, (name, kind, mass_kg, radius_m, distance_au) in enumerate(specs):
    distance_m = distance_au * AU_M
    planets_hand.append(
        Planet(
            name=name,
            kind=kind,
            mass_kg=mass_kg,
            radius_m=radius_m,
            distance_m=distance_m,
            phase_rad=i * 2.0 * math.pi / len(specs),
            period_s=k.period_s(distance_m),
            orbital_speed_mps=k.circular_speed_mps(distance_m),
        )
    )

system_hand = SolarSystem(star_hand, planets_hand)
sim_hand = Simulation(system_hand)

system_hand.state_m()


In [ ]:

for day in range(3):
    sim_hand.step(DAY_S)
    print(f"Day {day + 1}")
    for name, (x, y) in sim_hand.system.state_m().items():
        print(f"  {name}: x={x:.2e} m, y={y:.2e} m")


---
## 7. Exporting and reloading systems

HelioForge systems can be serialized to JSON for reuse.


In [ ]:

from helioforge.export import save_json, load_json

save_json(system_hand, "handmade_system.json")
system_loaded = load_json("handmade_system.json")

system_loaded.state_m()


---
## 8. Visualizing systems with CosmoView (Python)

CosmoView can render any HelioForge system directly from Python.


In [ ]:

from cosmoview.app import run

run(
    width=1100,
    height=800,
    fps_limit=60,
    system=system_hand,
)


---
## 9. CosmoView command-line usage

CosmoView can also be launched from a terminal.


```bash
# Built-in demos
cosmoview --mode solar
cosmoview --mode random

# Load a system exported from HelioForge
cosmoview --json handmade_system.json
```

---
## Summary

- HelioForge provides a deterministic heliocentric simulation model.
- Systems can be generated, handcrafted, exported, and reloaded.
- CosmoView visualizes any compatible HelioForge system interactively.
